In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from topo_lib import hatsugai, hamiltonians
import numpy as np

In [3]:
iq1=1; iq2=2
kx, ky = iq1*hamiltonians.b1+iq2*hamiltonians.b2

In [4]:
args_dict={ 't1' : -1, 'L1' : 1,  't2': 0, 'L2': 0 }
args_dict={ 't1' : -1, 'L1' : 0.28,  't2': 0.3, 'L2': 0.2 } 
args_dict={ 't1' : 0, 'L1' : 0,  't2': -1, 'L2': -1 }

In [5]:
args_dict

{'t1': 0, 'L1': 0, 't2': -1, 'L2': -1}

In [6]:
hamiltonians.Hamiltonian(kx,ky,**args_dict)

array([[ 0.+0.j,  2.-2.j, -2.-2.j],
       [ 2.+2.j,  0.+0.j,  2.-2.j],
       [-2.+2.j,  2.+2.j,  0.+0.j]])

In [7]:
hamiltonians.Kagome_Hamiltonian(iq1,iq2,**args_dict)

array([[ 0.+0.j, -2.+2.j, -2.-2.j],
       [-2.-2.j,  0.+0.j, -2.+2.j],
       [-2.+2.j, -2.-2.j,  0.+0.j]])

In [8]:
# Brilliun Zone Define and Allowed Pairs
N1 = 25
N2 = 25
q1_list = np.arange(0,N1) / N1 
q2_list = np.arange(0,N2) / N2 

In [9]:
q2_list

array([0.  , 0.04, 0.08, 0.12, 0.16, 0.2 , 0.24, 0.28, 0.32, 0.36, 0.4 ,
       0.44, 0.48, 0.52, 0.56, 0.6 , 0.64, 0.68, 0.72, 0.76, 0.8 , 0.84,
       0.88, 0.92, 0.96])

In [10]:
def build_U(vec1, vec2):
    inner_product = np.dot(vec1, vec2.conj())
    U = inner_product / np.abs(inner_product)
    return U

def latF(k_vec, Dk):

    H_k = hamiltonians.Hamiltonian

    E, aux = np.linalg.eig(H_k(*k_vec,**args_dict))
    idx = E.argsort()
    psi = aux[:,idx] # column vectors

    E, aux = np.linalg.eig(H_k(k_vec[0]+Dk[0], k_vec[1], **args_dict))
    idx = E.argsort()
    psiDx = aux[:,idx]

    E, aux = np.linalg.eig(H_k(k_vec[0], k_vec[1]+Dk[1], **args_dict))
    idx = E.argsort()
    psiDy = aux[:,idx]

    E, aux = np.linalg.eig(H_k(k_vec[0]+Dk[0], k_vec[1]+Dk[1], **args_dict))
    idx = E.argsort()
    psiDxDy = aux[:,idx]

    size=3
    U1x = np.zeros(size, dtype=complex)
    U2y = np.zeros(size, dtype=complex)
    U1y = np.zeros(size, dtype=complex)
    U2x = np.zeros(size, dtype=complex)

    for i in range(size):
        
        U1x[i] = build_U(psi[:,i], psiDx[:,i])
        
        U2y[i] = build_U(psi[:,i], psiDy[:,i])
        
        U1y[i] = build_U(psiDy[:,i], psiDxDy[:,i])
        
        U2x[i] = build_U(psiDx[:,i], psiDxDy[:,i])
        
    F12 = np.zeros(size, dtype=complex)
    F12 = np.log( U1x*U2x*1/U1y*1/U2y)
    # column vector
    return F12

In [11]:
size=3
LF = np.zeros(size, dtype=complex)
Sum = np.zeros(size, dtype=complex)
Chern = np.zeros(size, dtype=complex)
Q = np.zeros([N1,N2,2])
F21=np.zeros([N1,N2],dtype=complex) 

#Dx=Delta k_x, Dy=Delta k_y (N1=N2=25)
Dx = 0.1256637061435919;Dy = 0.14510394913873714
Dk = np.array([Dx,Dy], float)

for iq1, q1 in enumerate(q1_list):
    for iq2, q2 in enumerate(q2_list):
        Q[iq1,iq2,:] = q1*hamiltonians.b1+q2*hamiltonians.b2
        kx,ky=Q[iq1,iq2,:]
        k_vec = np.array([kx,ky], float)
        LF = latF(k_vec, Dk)
        Sum += LF
        F21[iq1,iq2] = LF[0]
Chern = Sum.imag/(2*np.pi)
print(Chern)

[-3.00000000e+00  1.23167118e-15  3.00000000e+00]


In [12]:
# Eigenvalues and Eigenenergies of Matrix
EEA, UUA = hatsugai.get_all_eigen(hamiltonians.Kagome_Hamiltonian, q1_list, q2_list, 3, **args_dict)

for bi in range(3):
    U1, U2 = hatsugai.get_U1_U2(bi,bi,UUA)

    F12 = hatsugai.get_F12(U1,U2)
    print(np.sum(F12.imag)/(2*np.pi)/8)

3.0
0.0
-3.0
